<a href="https://colab.research.google.com/github/gbr-guimaraes/Projeto-organizador-de-estudos/blob/main/Projeto_Cronograma_de_estudos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalação das bibliotecas

In [1]:
!pip install -q -U google-generativeai  # Instala a biblioteca google-generativeai
!pip install PyPDF2 -q                  # Instala a biblioteca PyPDF2

#Importação das dependencias

In [2]:
from google.colab import userdata
import google.generativeai as genai
import os
import PyPDF2
import ipywidgets as widgets
from IPython.display import display, Markdown
import textwrap

# Configurações

In [3]:
API_KEY = userdata.get('SECRET_KEY')  # Obtém a chave de API do Gemni
max_token = 1048576                   # Define o limite máximo de tokens para o modelo
list_command = "Apresente os cargos disponiveis neste edital em formato de lista simples,apenas texto, um elemento em cada linha, sem detalhes, textos de introdução ou caracteres especiais"
main_command = "A partir do texto acima, crie um cronograma de estudos personalizado, em formato de tabela Markdown, com os conteúdos gerais e específicos para o cargo de {role} para o estudante {student_name}, este aluno tem disponibilidade de estudar {hours} horas por dia, nos dias {week_days}"


# Configuração do modelo

In [4]:
genai.configure(api_key=API_KEY)          # Configura a API com a chave
model = genai.GenerativeModel('gemini-1.5-pro-latest')  # Define o modelo a ser usado
convo = model.start_chat(history=[])     # Inicia uma conversa com o modelo, mantendo o histórico vazio

# Função para extrair texto do PDF

In [5]:
def extract_pdf_pages(pathname: str) -> list[str]:
    parts = [f"--- START OF PDF ${pathname} ---"]       # Inicia a lista de partes com um cabeçalho
    with open(pathname, "rb") as pdf_file:              # Abre o arquivo PDF em modo binário
        pdf_reader = PyPDF2.PdfReader(pdf_file)         # Cria um objeto leitor de PDF
        num_pages = len(pdf_reader.pages)               # Obtém o número de páginas
        for page_num in range(num_pages):               # Itera sobre cada página
            page = pdf_reader.pages[page_num]           # Obtém a página atual
            page_text = page.extract_text()             # Extrai o texto da página
            if len(parts) + len(page_text) > max_token: # Verifica se a parte excede o limite de tokens
                # Divide a entrada em partes menores
                convo.send_message(parts)
                parts = []
            parts.append(f"--- PAGE {page_num} ---")    # Adiciona um cabeçalho para a página
            parts.append(page_text)                     # Adiciona o texto da página à lista de partes
    return parts


# Função para obter a lista de cargos disponíveis no edital

In [6]:
def role_list_generate():
    # Combina o texto do PDF com o comando para listar os cargos em uma única mensagem
    convo.send_message("".join(extract_pdf_pages("/content/edital.pdf")) + list_command)    # Envia a mensagem combinada ao modelo
    return convo.last.text.splitlines()

# Função para gerar um cronograma de estudos personalizado

In [7]:
def call_gemni(role: str, student_name: str, week_days: str, hours):
    convo = model.start_chat(history=[])
    convo.send_message("".join(extract_pdf_pages("/content/edital.pdf")) + "\n\n" + main_command.format(role=role, student_name=student_name, hours=hours, week_days=week_days))
    display(to_markdown(convo.last.text))                                      # Imprime o cronograma gerado pelo modelo


# Função para atualizar a lista de cargos disponíveis após o upload do arquivo

In [8]:
def handle_file_upload(change):
    uploaded_file = change['new']
    file_save()
    roles_list = []
    roles_list = role_list_generate() # Obtém a lista de cargos do modelo
    roles.options = roles_list

# Função para processar o clique no botão "Gerar"

In [9]:
def on_button_clicked(b):
    # Obtém os dias da semana selecionados pelo usuário
    checkboxes = [monday, tuesday, wednesday, thursday, friday, saturday, sunday]
    week_list = ""
    for checkbox in checkboxes:
        if checkbox.value:
            week_list += checkbox.description + ", "
    # Chama a função para gerar o cronograma com as informações do usuário
    call_gemni(roles.value, name.value, week_list, time_avaliable.value)

# Função para salvar o arquivo PDF carregado pelo usuário

In [10]:
def file_save():
    if file_upload.data != []:
      with open("./edital.pdf", "wb") as fp:
          fp.write(file_upload.data[0])

# Função para formatar a resposta da IA

In [11]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Criação dos widgets da interface do usuário

In [12]:
# Widget para upload do arquivo PDF
file_upload = widgets.FileUpload(accept='.pdf',multiple=False)
# Widgets para seleção dos dias da semana disponíveis para estudo
monday = widgets.Checkbox(value=False, description='Segunda-Feira', disabled=False, indent=False)
tuesday = widgets.Checkbox(value=False, description='Terça-Feira', disabled=False, indent=False)
wednesday = widgets.Checkbox(value=False, description='Quarta-Feira', disabled=False, indent=False)
thursday = widgets.Checkbox(value=False, description='Quinta-Feira', disabled=False, indent=False)
friday = widgets.Checkbox(value=False, description='Sexta-Feira', disabled=False, indent=False)
saturday = widgets.Checkbox(value=False, description='Sábado', disabled=False, indent=False)
sunday = widgets.Checkbox(value=False, description='Domingo', disabled=False, indent=False)
# Widget para definir o número de horas disponíveis por dia
time_avaliable = widgets.BoundedIntText(value=5, min=1, max=24, step=1, description='',disabled=False)
# Widget para inserir o nome do usuário
name = widgets.Text(value='', placeholder='Nome...', description='', disabled=False)
# Widget para selecionar o cargo desejado
roles = widgets.Dropdown(options=[], description='', disabled=False,)
# Botão "Gerar"
submit = widgets.ToggleButton(value=False, description='Gerar', button_style='', tooltip='Description', icon='check')


# Exibição dos widgets na interface e eventos

In [13]:
print("Faça o upload do seu edital:")
display(file_upload)
print("\nSeu Nome:")
display(name)
print("\nCargo desejado:")
display(roles)
print("\nDias disponiveis para estudo:")
display(monday, tuesday, wednesday, thursday, friday, saturday, sunday)
print('\nHoras disponiveis por dia:')
display(time_avaliable)
print("\n")
display(submit)
print("\n\n")

# Vincula as funções aos eventos dos widgets
file_upload.observe(handle_file_upload, names='value') # Chama a função quando um arquivo é carregado
submit.observe(on_button_clicked)                      # Chama a função quando o botão é clicado

Faça o upload do seu edital:


FileUpload(value={}, accept='.pdf', description='Upload')


Seu Nome:


Text(value='', placeholder='Nome...')


Cargo desejado:


...

BoundedIntText(value=5, max=24, min=1)

ToggleButton(value=False, description='Gerar', icon='check', tooltip='Description')

> ## Cronograma de Estudos Personalizado para Gabriel - Tecnologista em Informações Geográficas e Estatísticas (IBGE)
> 
> **Disponibilidade:** 6 horas por dia (Segunda, Terça, Quinta e Sexta)
> 
> **Formato:** 2 horas por eixo temático (3 eixos por dia) + Revisão
> 
> **Período:** 19/01/2024 até 04/05/2024 (Considerando a data limite para tirar dúvidas sobre os locais de prova)
> 
> **Observações:**
> 
> * Este cronograma é uma sugestão, adapte-o de acordo com suas necessidades e ritmo de aprendizado.
> * Inclua tempo para resolução de exercícios e simulados.
> * A revisão é crucial para a fixação do conteúdo.
> * Concentre-se nos eixos temáticos com maior peso para o cargo de Tecnologista em Informações Geográficas e Estatísticas - Especialidade Estatística (Eixo 5 - 55%).
> 
> | Dia da Semana | Horário | Eixo Temático (Conhecimentos Específicos) | Conteúdo (Conhecimentos Gerais) | Revisão |
> |---|---|---|---|---|
> | Segunda-Feira | 08:00 - 10:00 | **Eixo 5:** Apoio à Decisão, Inteligência Artificial e Métodos Quantitativos (Parte 1) | 1. Políticas Públicas |  |
> |  | 10:00 - 12:00 | **Eixo 3:** Gerência e Suporte da Tecnologia da Informação | 2. Desafios do Estado de Direito |  |
> |  | 12:00 - 14:00 | **Eixo 1:** Gestão Governamental e Governança Pública (Parte 1) | 3. Ética e Integridade | Simulado Conhecimentos Gerais 1 | 
> | Terça-Feira | 08:00 - 10:00 | **Eixo 5:** Apoio à Decisão, Inteligência Artificial e Métodos Quantitativos (Parte 2) | 4. Diversidade e Inclusão na Sociedade |  |
> |  | 10:00 - 12:00 | **Eixo 4:** Desenvolvimento de Software (Parte 1) | 5. Administração Pública Federal |  |
> |  | 12:00 - 14:00 | **Eixo 2:** Políticas Públicas | 6. Finanças Públicas | Simulado Conhecimentos Gerais 2 |
> | Quarta-Feira |  | **DESCANSO** |  | Revisão Geral |
> | Quinta-Feira | 08:00 - 10:00 | **Eixo 5:** Apoio à Decisão, Inteligência Artificial e Métodos Quantitativos (Parte 3) | Revisão Eixo 1: Gestão Governamental e Governança Pública |  |
> |  | 10:00 - 12:00 | **Eixo 4:** Desenvolvimento de Software (Parte 2) | Revisão Eixo 2: Políticas Públicas |  |
> |  | 12:00 - 14:00 | **Eixo 1:** Gestão Governamental e Governança Pública (Parte 2) | Revisão Eixo 3: Gerência e Suporte da Tecnologia da Informação | Simulado Conhecimentos Específicos 1 |
> | Sexta-Feira | 08:00 - 10:00 | Revisão Eixo 5: Apoio à Decisão, Inteligência Artificial e Métodos Quantitativos | Revisão Eixo 4: Desenvolvimento de Software |  |
> |  | 10:00 - 12:00 | **Eixo 2:** Políticas Públicas (Revisão e Aprofundamento) | Revisão Eixo 5: Apoio à Decisão, Inteligência Artificial e Métodos Quantitativos |  |
> |  | 12:00 - 14:00 | Resolução de Exercícios (Eixos 3, 4 e 5) |  | Simulado Conhecimentos Específicos 2 |
> | Sábado e Domingo |  | **DESCANSO** |  | Revisão Geral e Simulados |
> 
> **Conteúdo (Conhecimentos Específicos):** Seguir a ordem dos tópicos do Anexo IV do Edital.
> 
> **Recursos Adicionais:**
> 
> * Cursos online específicos para o cargo.
> * Livros e apostilas de concursos públicos.
> * Sites e blogs especializados em concursos públicos.
> * Grupos de estudos online e presenciais.
> 
> Lembre-se: disciplina, foco e persistência são fundamentais para o sucesso nos estudos. Boa sorte, Gabriel! 


> ## Cronograma de Estudos para Gabriel - Tecnologista em Informações Geográficas e Estatísticas (6 horas/dia - Seg, Ter, Qui, Sex)
> 
> **Objetivo:** Concurso Público Nacional Unificado - Bloco 2: Tecnologia, Dados e Informação.
> 
> **Disponibilidade:** 6 horas de estudo por dia, às Segundas, Terças, Quintas e Sextas-feiras.
> 
> **Período:** 19 de Janeiro a 04 de Maio de 2024 (15 semanas).
> 
> **Método:** Ciclo de estudos com foco em rotação de disciplinas e revisões periódicas.
> 
> **Recursos:** Materiais de estudo específicos para o concurso, legislação, exercícios e simulados.
> 
> **Observações:**
> 
> * Este cronograma é uma sugestão inicial e pode ser adaptado de acordo com as necessidades e progresso de Gabriel.
> * É importante realizar simulados periodicamente para avaliar o aprendizado e identificar pontos fracos.
> * Dedique tempo para revisões regulares dos conteúdos já estudados.
> * Priorize o descanso e o lazer para manter a saúde mental e física durante a preparação.
> 
> **Legenda:**
> 
> * **CG:** Conhecimentos Gerais
> * **E1:** Eixo Temático 1 - Gestão Governamental e Governança Pública
> * **E2:** Eixo Temático 2 - Políticas Públicas
> * **E3:** Eixo Temático 3 - Gerência e Suporte da Tecnologia da Informação
> * **E4:** Eixo Temático 4 - Desenvolvimento de Software
> * **E5:** Eixo Temático 5 - Apoio à Decisão, Inteligência Artificial e Métodos Quantitativos
> * **REV:** Revisão
> 
> | Semana | Segunda-Feira (3h) | Terça-Feira (3h) | Quinta-Feira (3h) | Sexta-Feira (3h) | Revisão (Final de semana) |
> |---|---|---|---|---|---|
> | 1 | CG: Políticas Públicas | E1: Planejamento e Gestão Estratégica | E2: O processo de elaboração de políticas | E3: Segurança da Informação | CG e E1 |
> | 2 | CG: Desafios do Estado de Direito | E1: Gestão de Projetos | E2: Implementação de políticas públicas | E3: Governança e Gestão de TI | CG e E2 |
> | 3 | CG: Ética e Integridade | E1: Gestão de Processos de Negócio | E2: Avaliação de políticas públicas | E3: Sistemas Operacionais | CG e E3 |
> | 4 | CG: Diversidade e Inclusão | E1: Gestão de Riscos | E2: Políticas de CT&I | E3: Redes de Computadores | CG e E1 |
> | 5 | CG: Administração Pública Federal | E1: Inovação na Gestão Pública | E2: Políticas de Governo Digital | E4: Programação e Estruturas de Dados | CG e E2 |
> | 6 | CG: Finanças Públicas | E1: Governo Eletrônico | E4: Programação Web | E4: Desenvolvimento para Plataformas Móveis | CG e E4 |
> | 7 | REV CG | E1: Comunicação na Gestão Pública | E4: Engenharia de Software | E4: Banco de Dados | E1 e E4 |
> | 8 | E2: REV | E1: Compras Governamentais | E4: Arquitetura e Tecnologias de Sistemas de Informação | E5: Big Data | E1 e E5 |
> | 9 | E3: REV | E1: Organização Sistêmica da Adm. Pública Federal | E5: Data Warehouse | E5: Descoberta de Conhecimento e Mineração de Dados | E3 e E5 |
> | 10 | E4: REV | E5: Aprendizado de Máquina | E5: Estatística | E5: Noções de Probabilidade | E4 e E5 | 
> | 11 | E5: REV | Simulado Geral | Análise do Simulado | Revisão Pontos Fracos Simulado |  Todos os Conteúdos |
> | 12 | CG: Revisão Geral | E1: Revisão Geral | E2: Revisão Geral | E3: Revisão Geral | E4 e E5 |
> | 13 | E4: Revisão Geral | E5: Revisão Geral | Simulado Geral | Análise do Simulado | Todos os Conteúdos |
> | 14 | Revisão Pontos Fracos Simulado | Revisão Legislação | Revisão Temas Mais Importantes | Descanso |  - |
> | 15 | Simulado Geral | Revisão Geral | Revisão Geral | Descanso | - | 
> 
> **Sucesso nos estudos, Gabriel!** 


ReadTimeout: HTTPConnectionPool(host='localhost', port=37009): Read timed out. (read timeout=60.0)